# 함수 정의하기

인스타그램 검색 결과 URL을 만들어서 접속하기 (insta_searching 함수 만들기)

In [2]:
#함수 작성
def insta_searching(word):  #word라는 매개변수를 받는 insta_searching 이라는 함수 생성
    url = 'https://www.instagram.com/explore/tags/' + word
    return url

첫번째 게시글 열기 (selet_first 함수 만들기)

In [3]:
#열린 크롬으로 개발자 도구 활용하여 첫번째 게시물 태그 확인 (<div class="_9AhH0"></div>)
#첫번째 게시물 찾아 클릭 함수 만들기

import time

def select_first(driver):
    first = driver.find_element_by_css_selector('div._9AhH0') #find_element_by_css_selector 함수를 사용해 요소 찾기
    first.click()
    time.sleep(3) #로딩을 위해 3초 대기

게시글 정보 가져오기 (get_content 함수 만들기)

In [4]:
#본문 내용, 작성 일시, 위치 정보 및 해시태그(#) 추출

import re

def get_content(driver):
    # 1. 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    # 2. 본문 내용 가져오기
    try:                                         #여러 태그중 첫번째([0]) 태그를 선택  
        content = soup.select('div.C4VMK > span')[0].text #첫 게시글 본문 내용이 <div class="C4VMK"> 임을 알 수 있다.
                                #태그명이 div, class명이 C4VMK인 태그 아래에 있는 span 태그를 모두 선택.
    except:
        content = ' ' 
        
    # 3. 본문 내용에서 해시태그 가져오기(정규표현식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content) 
        # content 변수의 본문 내용 중 #으로 시작하며, #뒤에 연속된 문자(공백이나 #, \ 기호가 아닌 경우)를 모두 찾아 tags 변수에 저장
    
    # 4. 작성 일자 가져오기
    try:
        date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10] #앞에서부터 10자리 글자
    except:
        date = ''

    # 5. 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1] 
    except:
        like = 0
        
    # 6. 위치 정보 가져오기
    try:
        place = soup.select('div.JF9hh')[0].text
    except:
        place = ''
    
    # 7. 수집한 정보 저장하기
    data = [content, date, like, place, tags]
    return data

다음 게시글 열기 (move_next 함수 만들기)

In [5]:
def move_next(driver):
    right = driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow') 
    right.click()
    time.sleep(3)

☆ 여러 게시글 정보 수집하기 (만든 함수들로 코딩하기)

In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# 1. 크롬으로 인스타그램 - '사당맛집' 검색
driver = webdriver.Chrome("C:\\Users\\KIM EUNKI\\anaconda3\\chromedriver.exe")
word = '사당맛집'
url = insta_searching(word)
driver.get(url) 
time.sleep(4)

# 2. 로그인 하기
login_section = '//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button'
driver.find_element_by_xpath(login_section).click()
time.sleep(3)


elem_login = driver.find_element_by_name("username")
elem_login.clear()
elem_login.send_keys('ekey_11_11')

elem_login = driver.find_element_by_name('password')
elem_login.clear()
elem_login.send_keys('test12345')

time.sleep(1)

xpath = """//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/button"""
driver.find_element_by_xpath(xpath).click()

time.sleep(4)

xpath1 = """//*[@id="react-root"]/section/main/div/div/div/div/button"""
driver.find_element_by_xpath(xpath1).click()
time.sleep(4)

# 3. 검색페이지 접속하기
driver.get(url)
time.sleep(4)

# 4. 첫번째 게시글 열기
select_first(driver)

# 5. 비어있는 변수(results) 만들기
results = []

# 여러 게시물 크롤링하기
target = 1000 #크롤링할 게시물 수
for i in range(target):
    data = get_content(driver) #게시물 정보 가져오기
    results.append(data)
    move_next(driver)
    
print(results[:2])

[['#당산오돌한번 꽂힌 이후로 주기적으로 가게 되는 듯 ‼️친구가 알아 온 사당맛집 넘 안땡겨서 🤣😂😅그냥 내가 좋아하는 익숙한 당산오돌로 (이기적😚🥰)친구도 다행히 좋아해땨.. 😍소금에 콕 찍어서 먹는 육즙 가득 꼬들살 🐷🍗💞허버허버 먹다가 입천장 까졌따뤼 흑흑,,⠀#사당맛집 #사당역맛집 #사당고기집 #사당맛집추천', '2020-05-26', '1,162', '사당역', ['#당산오돌한번', '#사당맛집', '#사당역맛집', '#사당고기집', '#사당맛집추천']], ['#얼쑤 #신토불이 족발에 막걸리 한잔!맛, 서비스, 위생, 청결, 규모 그 어디도 빠지지 않는 사당역 랜드마크! #사당맛집 #사당역맛집 #사당역모임장소 #사당회식장소 #방배동맛집 #방배맛집 #사당고기집', '2020-06-16', '1,311', '수향가 사당점', ['#얼쑤', '#신토불이', '#사당맛집', '#사당역맛집', '#사당역모임장소', '#사당회식장소', '#방배동맛집', '#방배맛집', '#사당고기집']]]


In [7]:
#1000개 추출해서 2개만 표시해보았음.
#추출한 데이터를 데이터프레임 results_df 변수에 저장.

import pandas as pd
results_df = pd.DataFrame(results)
results_df.columns = ['content', 'data', 'like', 'place', 'tags']

In [1]:
results_df.info()

NameError: name 'results_df' is not defined

In [9]:
results_df['tags']

0      [#당산오돌한번, #사당맛집, #사당역마...
1      [#얼쑤, #신토불이, #사당맛집, #사당역맛집, #사당역모임장소, #사당회식장소,...
2      [#사당맛집, #사당역맛집, #사당역모임...
3      [#사당맛집, #사당족발, #사당역맛집, ...
4      [#떡, #이수떡집, #정애맛담, #민속떡지...
                             ...                        
995    [#이수맛집, #사당맛집, #이수역맛집, #ᄋ...
996                                            [#카페좋아진다]
997    [#이태리레스토랑, #사당, #사당맛집, #사당점심, #사당소개팅, #사당술집, #...
998    [#100족발.., #사당맛집, #사당역맛집, #사당술집, #사당역술집, #사당역족...
999    [#아이스아메리카노, #KIMBO, #espresso, #방배역, #방배동, #서래...
Name: tags, Length: 1000, dtype: object

수집한 데이터 엑셀파일로 저장하기

In [10]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content', 'data', 'like', 'place', 'tags']
results_df.to_excel("C:\\Users\\KIM EUNKI\\Google 드라이브\\GitHub\\Practice\\Insta_Sadang.xlsx")